## Задача
Задача - реализовать функционал для анализа степени сходства между заранее известным/и текстом/ами и входной строкой. Степень сходства определить как размер пересечения множества 10 наиболее популярных неуникальных 4-х символьных слов исходного/ых текста/ов и множества всех 4-х символьных слов входной строки.

## Решение
Решение задачи будет состоять из трёх очевидных этапов: 
- генерация словаря слов исходного текста или текстов;
- генерация словаря слов входной строки;
- сравнение схожести строки и каждого из текстов.

Для генерации словаря слов использовалось представление текста в виде мешка слов (bag of words или bow). Мешок слов - это модель представления текста в виде множества слов с сохранением информации об их количестве.

## Реализация
В качестве языка реализации использован язык программирования python версии 3.8.

Для решения задачи был реализован класс SimilarityAnalyzer. В нём собраны функции (методы), с помощью которых можно выполнить каждый из этапов, описанных ранее. Использовать класс можно, если импортировать его функционал как часть модуля similarity_analyzer.

In [46]:
from similarity_analyzer import SimilarityAnalyzer
analyzer = SimilarityAnalyzer() # class to analize similarity of texts and input string

Входными данными для программы являются массив исходных текстов (corpus_of_texts) и входная строка (input_text).

In [47]:
# Input data
corpus_of_texts = ["zoot crocodiLes diLet azoots", "kloot tool stool teloots kept xentool"]
input_text = "dile kool tool"

Так же входными данными к программе можно принять не массив исходных текстов, а массив путей до текстовых файлов. 

In [48]:
corpus_of_texts = ["data/text1.txt", "data/text2.txt"]

Для дальнейшего корректного оперирования текстовыми файлами необходимо устанавливать аргумент source='file' в функции генерации мешка слов generate_bow. 

Итак, выполняем первый этап: генерируем мешки слов для каждого текста:

In [49]:
# Generate bag of words for each text
base_bows = []
for text in corpus_of_texts:
    bow = analyzer.generate_bow(text=text, source='file')
    base_bows.append(bow)

Второй этап: генерация мешка слов для входной строки. 

Следует передать дополнительный аргумент в функцию generate_bow : non_unique_words=False. Так мы получим сгененированный словарь всех слов в строке, а не только не-уникальных. В качестве source используем 'string', чтобы оперировать переменной input_text как строкой, а не как путём к текстовому файлу со строкой.

In [50]:
input_bow = analyzer.generate_bow(text=input_text, source='string', non_unique_words=False)

Третий этап: анализ схожести. 

Метод get_similarity возвращает словарь, где ключами являются порядковые номера текстов, а значения в словаре это степень схожести текста с входной строкой. Словарь отсортирован таким образом, что вначале стоят порядковые номера текстов, которые наиболее схожи со входной строкой.

In [51]:
# Check the level of similarity between input_text and each text from corpus_of_texts
similarity = analyzer.get_similarity(base_bows=base_bows, input_bow=input_bow)

print(similarity)

{0: 1, 1: 1}


Для определения наиболее схожего текста необходимо просто взять первый элемент словаря. А если корректнее - первые несколько элементов словаря с одинаковой схожестью.

In [52]:
most_similar_text_index, max_similarity_degree = list(similarity.items())[0]
for text_index, similarity_degree in similarity.items():
    if similarity_degree != max_similarity_degree:
        break
    print(text_index, ' ')

0  
1  


Стоит упомянуть, что у метода generate_bow есть опциональные аргументы, с помощью которых можно по разному генерировать bag of words (bow).

bow_size - максимальный размер сгенерированного словаря (default = 10)

word_size - размер слов (default = 4)

skip_spaces - пропускать символы пробела при генерации слов или нет (default = True)

non_unique_words - гененировать словарь только из неуникальных слов или нет (default = True).

У метода 'get_similarity' опциональный аргумент case_sensitive - определять степень схожести с учётом регистра символов в словах или без учета (default = True)




## Дополнительно 
Дополнительно решено было реализовать метод generate_bow_ngram, который использует модель n-gram для генерации bow. Такая модель позволяет учитывать не только информацию о числе вхождений слова, но и пространственную информацию - число вхождений словосочетаний из n слов.

Из входного текста извлекаются слова (последовательность символов) между которыми в тексте стоит пробел. Для генерации необходимо учитывать аргумент ngram_range, который задаёт диапазон количества слов в словосочетаниях. Например, устанавливая ngram_range=(1,2) мы сгенерируем bow из слов и словосочетаний из 2-х слов.

**Пример использования**

Для примера взяты два текста, которые можно классифицировать как "рецепт" и "научный текст" - data/text_receipt.txt и data/text_wiki.txt соответственно.

Предлагается сравнить степень схожести каждого текста со строкой "oven butter marzipan stir combine flour dough". Ожидаемый результат - степень схожести текста "рецепт" выше.

In [53]:
# Input data
corpus_of_texts = ["data/text_receipt.txt", "data/text_wiki.txt"]
input_text = "oven butter marzipan stir combine flour dough"

# Generate bag of words for each text
base_bows = []
for text in corpus_of_texts:
    bow = analyzer.generate_bow_ngram(text=text, source='file', bow_size=50, ngram_range=(1, 1))
    base_bows.append(bow)

# Check the level of similarity between input_text and each text from corpus_of_texts
input_bow = analyzer.generate_bow_ngram(text=input_text, source='string', non_unique_words=False)
similarity = analyzer.get_similarity(base_bows=base_bows, input_bow=input_bow)

print(similarity)

most_similar_text_index, max_similarity_degree = list(similarity.items())[0]
for text_index, similarity_degree in similarity.items():
    if similarity_degree != max_similarity_degree:
        break
    print(text_index, ' ')

{0: 4}
0  


Степень схожести каждого текста со строкой "information model methods reference example". Ожидаемый результат - степень схожести текста "научный текст" выше.

In [54]:
input_text = "information model methods reference example" 
input_bow = analyzer.generate_bow_ngram(text=input_text, source='string', non_unique_words=False)
similarity = analyzer.get_similarity(base_bows=base_bows, input_bow=input_bow)

print(similarity)

most_similar_text_index, max_similarity_degree = list(similarity.items())[0]
for text_index, similarity_degree in similarity.items():
    if similarity_degree != max_similarity_degree:
        break
    print(text_index, ' ')

{1: 3}
1  
